# Deployment Insights: Access via Trino

The easy way to access the deployment access database is through this notebook that connects to the Trino instance hosted in AWS. Please update the first cell with appropriate config details.

# Trino Setup
For the SQL queries to exexute in this notebook, you must have a Trino server running, connected to the Aerospike database via the Aerospike Trino Connector. 

Set the following Trino server parameters.

In [11]:
TRINO_IP = "<IP>"
TRINO_PORT = "<port>"
TRINO_USER = "<user>"
TRINO_PASSWORD = "<password>"

## Install Trino Client

In [4]:
%%sh
wget -nc -nv https://repo1.maven.org/maven2/io/trino/trino-cli/379/trino-cli-379-executable.jar
mv trino-cli-379-executable.jar trino
chmod +x trino

2022-05-06 00:23:01 URL:https://repo1.maven.org/maven2/io/trino/trino-cli/379/trino-cli-379-executable.jar [10238827/10238827] -> "trino-cli-379-executable.jar" [1]


## Trino Command
Define the environment variable for short form of the Trino command. 

You can also run the Trino command line tool in a separate shell tab.

In [ ]:
%env TRINO=./trino --server $TRINO_IP:$TRINO_PORT --catalog aerospike --schema test --output-format=TSV_HEADER
%env TRINO_VERTICAL=./trino --server $TRINO_IP:$TRINO_PORT --catalog aerospike --schema test --output-format=VERTICAL

# Trino SQL Queries

Show schemas (namespaces).

In [34]:
!$TRINO --execute "show schemas";

Schema
bar
information_schema
test


Show tables (sets).

In [35]:
!$TRINO --execute "show tables";

Table
__default
insights


Get the range of case numbers and timeframe in the database.

`
select min(case_num) as from_case, max(case_num) as to_case, 
       substr(min(timestamp),1,10) as from_date, substr(max(timestamp),1,10) as to_date 
from insights
`


In [ ]:
!$TRINO --execute "select min(case_num) as from_case, max(case_num) as to_case, substr(min(timestamp),1,10) as from_date, substr(max(timestamp),1,10) as to_date from insights";


Show a sample record.

In [ ]:
!$TRINO_VERTICAL --execute "select * from test.insights limit 1" ;


Get customers using feature 'index_on_device'.
```
select customer, cluster_name, features 
from insights 
where contains(cast(features as array(VARCHAR)),'index_on_device')
```

In [ ]:
!$TRINO --execute "select customer, cluster_name, features from insights where contains(cast(features as array(VARCHAR)),'index_on_device')" ;


Get customers using feature 'xdr_dest' and release after 5.x
```
select customer, features, server_release 
from insights 
where contains(cast(features as array(VARCHAR)),'xdr_dest') and regexp_like(server_release, '^5.*')
```

In [ ]:
!$TRINO --execute "select customer, features, server_release from insights where contains(cast(features as array(VARCHAR)),'xdr_dest') and regexp_like(server_release, '^5.*');" ;


Get the largest deployed cluster by each customer.
```
select customer, max(cluster_size) as max_cluster_size
rom insights
group by customer
```

In [ ]:
!$TRINO --execute "select customer, max(cluster_size) as max_cluster_size from insights group by customer" ;


Get the largest namespace by objects for each customer.
```
select customer, max(transform(cast(namespaces as array<map<varchar, varchar>>), entry->entry['objects'])) 
                 as max_ns_objects
from insights 
group by customer
```

In [ ]:
!$TRINO --execute "select customer, max(transform(cast(namespaces as array<map<varchar, varchar>>), entry->entry['objects'])) as max_ns_objects from insights group by customer" ;


Get customers with single-bin namespaces. Also get (name, storage, data_in_index) for each single-bin namespace.

`
select customer, case_num, single_bin_ns from (
    select customer, transform(
        filter(cast(namespaces as array<map<varchar,varchar>>), 
               ns->ns['single_bin']='true'),
        ns->(ns['name'],ns['storage_engine'],ns['data_in_index']) 
        ) as single_bin_ns 
    from insights)
where single_bin_ns != Array[]
order by customer, case_num desc;
`

In [ ]:
!$TRINO_VERTICAL --execute "select customer, case_num, cluster_name, single_bin_ns from (select customer, case_num,  cluster_name, transform(filter(cast(namespaces as array<map<varchar,varchar>>), ns->ns['single_bin']='true'),ns->(ns['name'],ns['storage_engine'],ns['data_in_index']) ) as single_bin_ns from insights) where single_bin_ns != Array[] order by customer, case_num desc;"


Get customers that have single-bin in-memory namespaces.

`
select customer, case_num, cluster_name, single_bin_mem_ns from (
    select customer,  case_num, cluster_name, 
      filter(
        transform(cast(namespaces as array<map<varchar,varchar>>), 
                  ns->map_filter(ns,(k,v)->k in 
                  ('name','single_bin','storage_engine','data_in_index','enable_xdr'))), 
                   ns->ns['single_bin']='true' and (ns['storage_engine'] = 'memory' 
                                                    or ns['data_in_index'] = 'true'))
        as single_bin_mem_ns 
    from insights) 
where single_bin_mem_ns != Array[]
order by customer, case_num desc;
`

In [ ]:
!$TRINO --execute "select customer, case_num, cluster_name, single_bin_mem_ns from (select customer, case_num, cluster_name, filter(transform(cast(namespaces as array<map<varchar,varchar>>), ns->map_filter(ns,(k,v)->k in ('name','single_bin','storage_engine','data_in_index','enable_xdr'))), ns->ns['single_bin']='true' and (ns['storage_engine'] = 'memory' or  ns['data_in_index'] = 'true')) as single_bin_mem_ns from insights) where single_bin_mem_ns != Array[] order by customer, case_num desc";


Get customers that have single-bin in-memory namespaces, with xdr_src flag to indicate use of XDR.

`
select customer, case_num, cluster_name, xdr_src, single_bin_mem_ns from (
    select customer, case_num, cluster_name, contains(cast(features as array<varchar>), 
                                                     'xdr_src') as xdr_src, 
      filter(
        transform(cast(namespaces as array<map<varchar,varchar>>), 
                  ns->map_filter(ns,(k,v)->k in 
                  ('name','single_bin','storage_engine','enable_xdr','data_in_index))), 
                    ns->ns['single_bin']='true' and (ns['storage_engine'] = 'memory' or 
                                                     ns['data_in_index'] = 'true' ))
        as single_bin_mem_ns 
    from insights) 
where single_bin_mem_ns != Array[]
order by customer, case_num desc;
`

In [ ]:
!$TRINO --execute "select case_num, customer, cluster_name, xdr_src, single_bin_ns from (select case_num, customer, cluster_name, contains(cast(features as array<varchar>), 'xdr_src') as xdr_src, filter(transform(cast(namespaces as array<map<varchar,varchar>>), ns->map_filter(ns,(k,v)->k in ('name','single_bin','storage_engine','enable_xdr','data_in_index'))), ns->ns['single_bin']='true' and (ns['storage_engine'] = 'memory' or ns['data_in_index'] = 'true')) as single_bin_ns from insights) where single_bin_ns != Array[] order by customer, case_num desc";


Get features used in each deployed cluster.

`
select customer, cluster_name, max(timestamp) as last, features 
from insights 
group by customer, cluster_name, timestamp, features 
order by customer, cluster_name;
`


In [ ]:
!$TRINO --execute "select customer, cluster_name, max(timestamp) as last, features from insights group by customer, cluster_name, timestamp, features order by customer, cluster_name;"


# Programmatic Access via Trino
Install the trino package if not already installed to enable Python access by running the following cell.

In [46]:
#!pip install trino


In [ ]:
import trino
conn = trino.dbapi.connect(
    host=TRINO_IP,
    port=TRINO_PORT,
    user='admin',
    catalog='aerospike',
    schema='test'
)
cur = conn.cursor()
cur.execute('select * from insights limit 1')
rows = cur.fetchall()
print(rows)